In [ ]:
from google.colab import files
uploaded = files.upload()


Saving traffic_accident_ml_dataset.csv to traffic_accident_ml_dataset.csv


In [ ]:
import pandas as pd

df = pd.read_csv("traffic_accident_ml_dataset.csv")
print(df.isnull().sum())


Month                        0
Location                     0
Total_Accidents              0
Fatal_Accidents              0
Injury_Accidents             0
Material_Damage_Accidents    0
Deaths                       0
Injuries                     0
Driver_Fault                 0
Pedestrian_Fault             0
Vehicle_Type                 0
Daylight                     0
is_fatal                     0
dtype: int64


In [ ]:
import numpy as np

# ' ' veya boş string olanları NaN olarak değerlendir
df_clean = df.replace(['', ' ', 'unknown', '?'], np.nan)
print(df_clean.isnull().sum())


Month                        0
Location                     0
Total_Accidents              0
Fatal_Accidents              0
Injury_Accidents             0
Material_Damage_Accidents    0
Deaths                       0
Injuries                     0
Driver_Fault                 0
Pedestrian_Fault             0
Vehicle_Type                 0
Daylight                     0
is_fatal                     0
dtype: int64


In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize edilecek sütunlar
num_cols = ['Total_Accidents', 'Fatal_Accidents', 'Injury_Accidents',
            'Material_Damage_Accidents', 'Deaths', 'Injuries',
            'Driver_Fault', 'Pedestrian_Fault']

# StandardScaler'ı başlat
scaler = StandardScaler()

# Sadece bu sütunlara scale uygula
df_scaled = df.copy()
df_scaled[num_cols] = scaler.fit_transform(df[num_cols])

# Kontrol için ilk 5 satıra bakalım
df_scaled[num_cols].head()


,Total_Accidents,Fatal_Accidents,Injury_Accidents,Material_Damage_Accidents,Deaths,Injuries,Driver_Fault,Pedestrian_Fault
0,1.096947,0.040656,0.965041,1.018834,-0.813733,0.993992,-1.612452,1.612452
1,-0.957822,-1.422952,-1.015337,-0.979364,-0.116248,-1.009744,-0.124035,0.124035
2,0.711678,-0.447214,0.870738,0.796812,-1.511219,0.804960,0.620174,-0.620174
3,-1.022034,-0.691148,-0.977615,-1.008967,0.232495,-0.990841,1.364382,-1.364382
4,1.161159,1.504264,1.153649,1.166849,0.581238,1.183023,-0.868243,0.868243


In [9]:
# One-Hot Encoding uygulanacak sütunlar
cat_cols = ['Month', 'Location', 'Vehicle_Type', 'Daylight']

# One-hot encoding işlemi
df_final = pd.get_dummies(df_scaled, columns=cat_cols)

# Kontrol: kaç sütun oldu ve ilk 5 satır
print("Yeni sütun sayısı:", df_final.shape[1])
df_final.head()

Yeni sütun sayısı: 17


,Total_Accidents,Fatal_Accidents,Injury_Accidents,Material_Damage_Accidents,Deaths,Injuries,Driver_Fault,Pedestrian_Fault,is_fatal,Month_Dec,Month_Feb,Month_May,Location_Non-Urban,Location_Urban,Vehicle_Type_Car,Daylight_Day,Daylight_Night
0,1.096947,0.040656,0.965041,1.018834,-0.813733,0.993992,-1.612452,1.612452,1,False,False,True,False,True,True,True,False
1,-0.957822,-1.422952,-1.015337,-0.979364,-0.116248,-1.009744,-0.124035,0.124035,1,False,False,True,True,False,True,False,True
2,0.711678,-0.447214,0.870738,0.796812,-1.511219,0.804960,0.620174,-0.620174,0,False,True,False,False,True,True,True,False
3,-1.022034,-0.691148,-0.977615,-1.008967,0.232495,-0.990841,1.364382,-1.364382,1,False,True,False,True,False,True,False,True
4,1.161159,1.504264,1.153649,1.166849,0.581238,1.183023,-0.868243,0.868243,1,True,False,False,False,True,True,True,False


In [12]:
from sklearn.model_selection import train_test_split

# Özellikler (X) ve hedef (y)
X = df_final.drop("is_fatal", axis=1)
y = df_final["is_fatal"]

# %80 eğitim, %20 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Boyut kontrolü
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)


X_train: (4, 16)
X_test: (2, 16)


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Modeli eğit
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Tahmin yap
y_pred_logreg = logreg.predict(X_test)

# Sonuçları yazdır
print("📌 Logistic Regression Results:")
print(confusion_matrix(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))


📌 Logistic Regression Results:
[[2]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [14]:
from sklearn.ensemble import RandomForestClassifier

# Modeli eğit
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Tahmin yap
y_pred_rf = rf.predict(X_test)

# Sonuçları yazdır
print("🌲 Random Forest Results:")
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


🌲 Random Forest Results:
[[2]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [15]:
from sklearn.neighbors import KNeighborsClassifier

# Modeli oluştur ve eğit
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Tahmin yap
y_pred_knn = knn.predict(X_test)

# Sonuçları yazdır
print("👟 KNN Results:")
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn))


👟 KNN Results:
[[2]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [16]:
from sklearn.svm import SVC

# Modeli oluştur ve eğit
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# Tahmin yap
y_pred_svm = svm.predict(X_test)

# Sonuçları yazdır
print("🧠 SVM Results:")
print(confusion_matrix(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


🧠 SVM Results:
[[2]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
